In [148]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import OrbitPlotter3D
import copy
import CustomManeuvres
from poliastro.twobody.thrust.change_a_inc import change_a_inc
from poliastro._math.linalg import norm
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [149]:
# Define the two coplanar debris
a = 3000+6371 << u.km
ecc = 0 << u.one
inc = 0 << u.deg
raan = 0 << u.deg
argp = 0 << u.deg
nu = 10 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 5000+6371 << u.km
inc = 45 << u.deg
nu = 60 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

In [128]:
print(deb_1.epoch.iso)
print(deb_2.epoch.iso)
print(otv.epoch.iso)

2000-01-01 12:00:00.000
2000-01-01 12:00:00.000
2000-01-01 12:00:00.000


In [135]:
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [150]:
# Calculate inc change with constant radius (no hohmann)
orbit_i = otv
orbit_f = deb_2

# Propagate to thrust location (0, 180)
thrust_location = 0 * u.deg if orbit_i.nu <= 0 else 179.999 * u.deg
time_to_thrust = orbit_i.time_to_anomaly(thrust_location)
print('currrent_anomaly', orbit_i.nu)
print('thrust_location', thrust_location)
print('time_to_thrust', time_to_thrust)
orbit_i = orbit_i.propagate_to_anomaly(thrust_location)

# Calculate the thrust value
v = norm(orbit_i.v << u.m / u.s)
inc_i = orbit_i.inc << u.rad
inc_f = orbit_f.inc << u.rad
inc_delta = inc_f - inc_i
thrust_norm = 2*v*np.sin((inc_delta << u.rad)/2)

print('thrust_norm', thrust_norm)

# Calculate the thrust vector
y_thrust = np.sin(inc_delta/2)*thrust_norm
z_thrust = -np.cos(inc_delta/2)*thrust_norm

print('y_thrust', y_thrust)
print('z_thrust', z_thrust)

thrust_vector = np.array([0,y_thrust,z_thrust]) * u.m / u.s

man = Maneuver(
        (time_to_thrust.decompose(), thrust_vector.decompose())
    )

print(man.impulses)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(man)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)



currrent_anomaly 10.0 deg
thrust_location 179.999 deg
time_to_thrust 4263.180818936897 s
thrust_norm 4991.666653212488
y_thrust 1910.2281280737152
z_thrust -4611.698654022133
((<Quantity 4263.18081894 s>, <Quantity [    0.        ,  1910.22812807, -4611.69865402] m / s>),)
otv inc 44.999999998192656 deg
deb_2 inc 45.0 deg


In [152]:
import CustomManeuvres
inc_change = CustomManeuvres.simple_inc_change(otv, deb_2)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

currrent_anomaly 1.5707839854470933 rad
thrust_location 179.999 deg
time_to_thrust 2256.9987021710513 s
thrust_norm 2.0572726326371667e-07 m / s
y_thrust 3.2447407509483956e-18 m / s
z_thrust -2.0572726326371667e-07 m / s
otv inc 44.99999999819266 deg
deb_2 inc 45.0 deg


In [123]:
# Calculate a basic homnann
import CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)
hoh_phas

Target Delta:  23.21465456106945 deg
current_delta :  330.0007071071426 deg
dist before:  306.78605254607317 deg
dist after:  306.78605254607317 deg
t_1:  30546.255748921016 s
orbit_i.nu:  -131.9363565966929 deg
new delta:  23.21567610790731 deg


Number of impulses: 2, Total cost: 0.599873 km / s

In [18]:
hoh_phas.impulses

((<Quantity 33931.51750328 s>,
  <Quantity [297.99784075,  74.61079575,   0.        ] m / s>),
 (<Quantity 5255.47453664 s>,
  <Quantity [-283.91309118,  -71.08434613,    0.        ] m / s>))

In [124]:
# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)



In [34]:
print("OTV ANGLE: ", otv.nu.to(u.deg))
print("DEB2 ANGLE: ", deb_2.nu.to(u.deg))
print('difference: ', (otv.nu - deb_2.nu).to(u.deg))
print('OTV.a: ', otv.a)
print('Deb2.a: ', deb_2.a)

OTV ANGLE:  -41.493535113462215 deg
DEB2 ANGLE:  -41.49353511346191 deg
difference:  -3.053332494204976e-13 deg
OTV.a:  11370.999999999989 km
Deb2.a:  11371.0 km
